In [ ]:
%pip install rioxarray
%pip install xarray
%pip install earthengine-api rasterio
%pip install geopandas
%pip install gdal==$(gdal-config --version) geemap
%pip install setuptools

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='btech-project-443813')

### Donwload LST Composite 8 Day (Not Working)

In [ ]:
import ee
import geemap
import os
from datetime import datetime, timedelta

def get_composite_lst(start_date, roi):
    """Get 8-day composite LST"""
    try:
        ee_start_date = ee.Date(start_date.strftime('%Y-%m-%d'))
        ee_end_date = ee_start_date.advance(8, 'day')
        
        dataset = ee.ImageCollection('MODIS/061/MOD11A2') \
            .filterDate(ee_start_date, ee_end_date) \
            .filterBounds(roi)
        
        lst_image = dataset.mean()  # Use mean to get composite
        
        if dataset.size().getInfo() == 0:
            raise Exception("No LST data available for this period")
        
        qc = lst_image.select('QC_Day').toUint8()
        
        good_quality = qc.bitwiseAnd(ee.Number(3)).eq(0)
        
        return lst_image.select('LST_Day_1km').multiply(0.02).subtract(273.15).updateMask(good_quality).clip(roi)
    
    except Exception as e:
        print(f"Error processing LST composite for {start_date.strftime('%Y-%m-%d')}: {str(e)}")
        raise

def download_composite_lst(start_date, roi, output_directory):
    """Download 8-day composite LST"""
    try:
        date_str = start_date.strftime('%Y-%m-%d')
        lst = get_composite_lst(start_date, roi)
        filename = os.path.join(output_directory, f'lst_delhi_composite_{date_str}.tif')
        
        region = roi.getInfo()['coordinates'][0]
        
        geemap.ee_export_image(
            lst,
            filename=filename,
            scale=1000,  # 1km resolution
            region=region,
            crs='EPSG:4326',
            file_per_band=False
        )
        
        if os.path.exists(filename):
            file_size = os.path.getsize(filename)
            print(f"Successfully downloaded LST composite for {date_str} (Size: {file_size/1024:.1f} KB)")
            return True
        else:
            print(f"Failed to download LST composite for {date_str}")
            return False
            
    except Exception as e:
        print(f"Error downloading LST composite for {date_str}: {str(e)}")
        return False

def main():
    ee.Initialize()
    
    roi = ee.Geometry.Rectangle([
        76.8,
        28.4,
        77.6,
        28.9 
    ])
    
    start_date = datetime(2000, 1, 1)
    end_date = datetime(2023, 12, 31)
    
    output_directory = r"/home/stormej/dev/rainscale/data/lst/lst_tif_temp"
    os.makedirs(output_directory, exist_ok=True)
    
    current_date = start_date
    successful_downloads = 0
    failed_downloads = 0
    
    print(f"\nStarting download of 8-day composite LST data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")
    
    while current_date <= end_date:
        try:
            print(f"\nProcessing {current_date.strftime('%Y-%m-%d')} to {(current_date + timedelta(days=7)).strftime('%Y-%m-%d')}")
            
            if download_composite_lst(current_date, roi, output_directory):
                successful_downloads += 1
            else:
                failed_downloads += 1
                
        except Exception as e:
            print(f"Error processing {current_date.strftime('%Y-%m-%d')}: {str(e)}")
            failed_downloads += 1
            
        # Move to the next 8-day period
        current_date += timedelta(days=8)
    
    print("\nDownload Summary:")
    print(f"Successfully downloaded: {successful_downloads} periods")
    print(f"Failed downloads: {failed_downloads} periods")
    print("Process completed.")

if __name__ == "__main__":
    main()


### Create Monthly LST

In [12]:
import os
import glob
import numpy as np
import rasterio
from datetime import datetime
from collections import defaultdict

def fill_nan_values(data_array, method='nearest_neighbor', max_search_radius=100):
    filled_array = np.copy(data_array)
    if not np.isnan(filled_array).any():
        return filled_array
    
    if method == 'global_mean':
        valid_mean = np.nanmean(filled_array)
        filled_array = np.where(np.isnan(filled_array), valid_mean, filled_array)
    
    elif method == 'nearest_neighbor':
        nan_mask = np.isnan(filled_array)
        nan_indices = np.where(nan_mask)
        for i, j in zip(nan_indices[0], nan_indices[1]):
            value_set = False
            for radius in range(1, max_search_radius + 1):
                i_min, i_max = max(0, i - radius), min(filled_array.shape[0], i + radius + 1)
                j_min, j_max = max(0, j - radius), min(filled_array.shape[1], j + radius + 1)
                neighborhood = filled_array[i_min:i_max, j_min:j_max]
                if np.count_nonzero(~np.isnan(neighborhood)) > 0:
                    filled_array[i, j] = np.nanmean(neighborhood)
                    value_set = True
                    break
            if not value_set and not np.isnan(filled_array[~np.isnan(filled_array)]).all():
                filled_array[i, j] = np.nanmean(filled_array[~np.isnan(filled_array)])
    return filled_array

def calculate_lst_monthly_averages(input_dir, output_dir, start_date=None, end_date=None):
    os.makedirs(output_dir, exist_ok=True)
    if start_date:
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if end_date:
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    lst_files = glob.glob(os.path.join(input_dir, 'lst_delhi_composite_*.tif'))
    print(f"Found {len(lst_files)} LST files")
    
    monthly_files = defaultdict(list)
    for file_path in lst_files:
        filename = os.path.basename(file_path)
        date_str = filename.split('_')[-1].split('.')[0]
        date = datetime.strptime(date_str, '%Y-%m-%d')
        
        if start_date and date < start_date:
            continue
        if end_date and date > end_date:
            continue
            
        month_key = date.strftime('%Y-%m')
        monthly_files[month_key].append(file_path)
    
    print(f"Processing {len(monthly_files)} months")
    
    for month_key, files in monthly_files.items():
        print(f"Processing {month_key} with {len(files)} files")
        
        with rasterio.open(files[0]) as src:
            metadata = src.profile
            data_shape = src.shape
        
        data_sum = np.zeros(data_shape)
        data_count = np.zeros(data_shape)
        
        for file_path in files:
            with rasterio.open(file_path) as src:
                data = src.read(1).astype(float)
                if src.nodata is not None:
                    mask = (data == src.nodata)
                    data[mask] = np.nan
                
                valid_mask = ~np.isnan(data)
                data_sum[valid_mask] += data[valid_mask]
                data_count[valid_mask] += 1
        
        with np.errstate(divide='ignore', invalid='ignore'):
            monthly_avg = np.where(data_count > 0, data_sum / data_count, np.nan)
        
        # Fill NaN values in the monthly average
        monthly_avg = fill_nan_values(monthly_avg, method='nearest_neighbor')
        
        metadata.update({
            'dtype': rasterio.float32,
            'nodata': np.nan
        })
        
        output_file = os.path.join(output_dir, f'lst_delhi_{month_key}.tif')
        with rasterio.open(output_file, 'w', **metadata) as dst:
            dst.write(monthly_avg.astype(rasterio.float32), 1)
        
        print(f"  Saved {output_file}")
    
    print(f"Successfully processed {len(monthly_files)} monthly averages")

input_dir = "/home/stormej/dev/rainscale/data/lst/lst_tif"
output_dir = "/home/stormej/dev/rainscale/data/lst/lst_monthly_avg"

calculate_lst_monthly_averages(input_dir, output_dir)

Found 1055 LST files
Processing 278 months
Processing 2000-02 with 2 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-02.tif
Processing 2000-03 with 4 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-03.tif
Processing 2000-04 with 4 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-04.tif
Processing 2000-05 with 3 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-05.tif
Processing 2000-06 with 4 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-06.tif
Processing 2000-07 with 4 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-07.tif
Processing 2000-08 with 4 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-08.tif
Processing 2000-09 with 4 files
  Saved /home/stormej/dev/rainscale/data/lst/lst_monthly_avg/lst_delhi_2000-09.tif
Processing 2000-10 with 4 files
  Sav

### Resample LST to 0.25 degree

In [13]:
import os
import rasterio
from rasterio.enums import Resampling
from affine import Affine

input_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_avg"
output_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_resampled_0.25"
os.makedirs(output_dir, exist_ok=True)

crs = 'EPSG:4326'
shape = (3, 5)
transform = Affine(0.25, 0, 76.5, 0, -0.25, 28.9)

for file in os.listdir(input_dir):
    if file.endswith('.tif'):
        path = os.path.join(input_dir, file)
        with rasterio.open(path) as src:
            resample = src.read(
                out_shape = (1, shape[0], shape[1]),
                resampling= Resampling.average,
            )
        parts = file.split('_')
        month = parts[2].replace('.tif', '')
        
        output_filename = f"lst_resampled_{month}.tif"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype=resample.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
            dst.write(resample[0], 1)
        
print("All LST images resampled successfully.")

All LST images resampled successfully.


### Resample LST to 0.01 degree

In [14]:
import os
import rasterio
from rasterio.enums import Resampling
from affine import Affine


bounds = {
    'min_lon': 76.5,  # Western boundary of Delhi
    'max_lon': 77.5,  # Eastern boundary of Delhi
    'min_lat': 28.3,  # Southern boundary of Delhi
    'max_lat': 28.9   # Northern boundary of Delhi
}

input_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_avg"
output_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_resampled_0.01"
os.makedirs(output_dir, exist_ok=True)

crs = 'EPSG:4326'
width = int((bounds['max_lon'] - bounds['min_lon']) / 0.01)  # 100 pixels
height = int((bounds['max_lat'] - bounds['min_lat']) / 0.01)  # 60 pixels
shape = (height, width)
transform = Affine(0.01, 0, bounds['min_lon'], 0, -0.01, bounds['max_lat'])


for file in os.listdir(input_dir):
    if file.endswith('.tif'):
        path = os.path.join(input_dir, file)
        with rasterio.open(path) as src:
            resample = src.read(
                out_shape = (1, shape[0], shape[1]),
                resampling= Resampling.average,
            )
        parts = file.split('_')
        month = parts[2].replace('.tif', '')
        
        output_filename = f"lst_resampled_{month}.tif"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype=resample.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
            dst.write(resample[0], 1)
        
print("All LST images resampled successfully.")

All LST images resampled successfully.


### Resample LST to 0.1 degree

In [1]:
import os
import rasterio
from rasterio.enums import Resampling
from affine import Affine


bounds = {
    'min_lon': 76.5,  # Western boundary of Delhi
    'max_lon': 77.5,  # Eastern boundary of Delhi
    'min_lat': 28.3,  # Southern boundary of Delhi
    'max_lat': 28.9   # Northern boundary of Delhi
}

input_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_avg"
output_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_resampled_0.1"
os.makedirs(output_dir, exist_ok=True)

crs = 'EPSG:4326'
width = int((bounds['max_lon'] - bounds['min_lon']) / 0.1)  # 10 pixels
height = int((bounds['max_lat'] - bounds['min_lat']) / 0.1)  # 6 pixels
shape = (height, width)
transform = Affine(0.1, 0, bounds['min_lon'], 0, -0.1, bounds['max_lat'])


for file in os.listdir(input_dir):
    if file.endswith('.tif'):
        path = os.path.join(input_dir, file)
        with rasterio.open(path) as src:
            resample = src.read(
                out_shape = (1, shape[0], shape[1]),
                resampling= Resampling.average,
            )
        parts = file.split('_')
        month = parts[2].replace('.tif', '')
        
        output_filename = f"lst_resampled_{month}.tif"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype=resample.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
            dst.write(resample[0], 1)
        
print("All LST images resampled successfully.")

All LST images resampled successfully.
